### Logistic Classifier for CIFAR-10

In [1]:
import matplotlib.pyplot as plt
from data_loader import load_training, load_testing, load_validation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

from Cifar10 import Cifar10

In [2]:
# device configuration
# device = torch.device('mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

print(f'Using device {device}')

# consitency in training
torch.manual_seed(1234)
random.seed(1234)

Using device cuda


In [3]:
# Load data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = Cifar10(train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = Cifar10(train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Hyperparameters for LogReg model:
- solver
- loss function
- epochs
- learning rate

Solvers: 
- Adam
- SGD

Loss functions:
- L1 Loss
- MSE Loss (L2)
- CrossEntropyLoss


In [4]:
# define Logistic Classifier

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.log_layer = nn.Sequential(
            nn.Linear(3 * 32 * 32, 1024),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = self.log_layer(x)
        return x


net = Net().to(device)

In [5]:
def evaluate(loader) -> float:
    'evaluates model accuracy on loader data'
    # Evaluate
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the {len(testset)} test images: {100 * correct // total} %')
    return correct / total

In [6]:
def train_model(criterion, optimizer, num_epochs: int, frequency: int = 2000) -> tuple[list, list, list]:
    'trains model with parameters and returns loss, tr_accuracy, te_accuracy'

    print('starting training')

    # train the model 
    for epoch in range(num_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            # backward + optimze
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

    print('Finished Training')

In [7]:

# Hyper Parameters
num_epochs = 1
learning_rate = 0.001

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

train_model(criterion, optimizer, num_epochs)

evaluate(testloader)

starting training
[1,  2000] loss: 3.662
[1,  4000] loss: 3.261
[1,  6000] loss: 3.093
[1,  8000] loss: 3.014
[1, 10000] loss: 2.863
Finished Training
Accuracy of the network on the 10000 test images: 35 %


0.3567